# Get GAV estimation for resonance radical

In [2]:
import sys
sys.path.insert(0,"/home/gridsan/hwpang/Software/RMG-Py/")

import random
import os
import time
import math
from copy import deepcopy
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm

from utils import load_thermo_lib_by_path, generate_thermo
from parameters import Ts

from rmgpy.data.thermo import ThermoDatabase, ThermoLibrary, ThermoData, remove_thermo_data, add_thermo_data, NASA
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy import settings
from rmgpy import constants

# Load HBI

In [3]:
path = "../data/hbi.csv"
hbi_df = pd.read_csv(path)
hbi_df

,radical_smiles,radical_resonance_smiles,closed_shell_smiles,closed_shell_H298 (kcal/mol),closed_shell_S298 (cal/mol/K),closed_shell_Cp300 (cal/mol/K),closed_shell_Cp400 (cal/mol/K),closed_shell_Cp500 (cal/mol/K),closed_shell_Cp600 (cal/mol/K),closed_shell_Cp800 (cal/mol/K),...,radical_num_resonance,HBI_H298 (kcal/mol),HBI_Sint298 (cal/mol/K),HBI_Cp300 (cal/mol/K),HBI_Cp400 (cal/mol/K),HBI_Cp500 (cal/mol/K),HBI_Cp600 (cal/mol/K),HBI_Cp800 (cal/mol/K),HBI_Cp1000 (cal/mol/K),HBI_Cp1500 (cal/mol/K)
0,[O]C(=O)OC(O)(O)O,[O]C(=O)OC(O)(O)O,O=C(O)OC(O)(O)O,-272.617591,98.406448,31.562314,36.827897,41.095602,44.234417,48.821415,...,1,49.103465,-4.589644,2.364565,1.559776,0.412894,-0.172027,-1.041685,-1.683176,-3.705453
1,[O]C(O)(O)OC(=O)O,[O]C(O)(O)OC(=O)O,O=C(O)OC(O)(O)O,-272.617591,98.406448,31.562314,36.827897,41.095602,44.234417,48.821415,...,1,49.194200,-7.808181,0.968866,0.741762,0.402841,0.476043,0.534574,0.412253,-1.394250
2,O=C(O)O[C](O)O,O=C(O)O[C](O)O,O=C(O)OC(O)O,-231.363843,94.127419,27.214474,31.965583,36.032333,39.309560,44.059044,...,1,44.040974,-4.610350,1.758768,2.587490,2.714389,2.346110,1.178471,0.282207,-0.997631
3,CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)O[O],CC(=O)OC(OO)C(=O)C(O)OO,-234.219312,145.598548,52.706616,62.438776,71.442505,78.070363,87.443365,...,1,48.558735,-12.393484,3.863088,3.589450,2.161239,1.702481,1.262146,-0.928170,-3.176204
4,O=[C]OC(O)(O)O,O=[C]OC(O)(O)O,O=COC(O)(O)O,-216.499044,92.684651,26.935163,30.962696,34.637667,37.592447,42.375430,...,1,36.001481,-4.835845,5.443029,5.881647,4.696512,3.708683,1.768296,0.144000,-3.234417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862,C1=C[CH]C=1,[C]1=CC=C1,C1=CC=C1,104.333500,57.818020,15.504000,19.920000,23.474000,26.261000,30.329000,...,2,51.773082,3.435468,-0.850889,-1.431243,-1.911856,-2.405821,-3.006968,-3.449835,-4.556360
2863,C=C=C1C=[C]C1,C=C=C1C=[C]C1,C=C=C1C=CC1,91.366874,50.394958,22.416919,29.489266,35.000461,39.185145,45.162263,...,1,66.366211,25.815240,0.258337,-1.155381,-2.024215,-2.373362,-2.314548,-2.090808,-2.168527
2864,[CH2]C1=CC#CC1,[CH2]C1=CC#CC1,CC1=CC#CC1,131.227000,72.956831,23.409000,30.017000,34.619000,39.828000,46.529000,...,3,32.271344,0.823844,-0.802362,-1.521882,-1.227900,-2.500041,-3.127836,-3.711522,-2.960733
2865,[CH2]C1=CC#CC1,C=C1[CH]C#CC1,C=C1CC#CC1,136.767000,72.435147,22.819000,29.017000,34.189000,38.448000,45.419000,...,3,26.731344,0.773844,-0.212362,-0.521882,-0.797900,-1.120041,-2.017836,-2.931522,-2.570733


# Estimate GAV for resonance radicals

In [5]:
# load thermo database

thermo_database = ThermoDatabase()
thermo_database.load_groups(os.path.join(settings["database.directory"], "thermo", "groups"))

In [6]:
generate_thermo(thermo_database, "[O]C(=O)OC(O)(O)O", resonance=False) # we want to know the GAV estimation for the exact resonance radical

/home/gridsan/hwpang/Software/RMG-Py/rmgpy/rmg/reactors.py:52: RuntimeWarning: Unable to import Julia dependencies, original error: No module named 'julia'
  warnings.warn("Unable to import Julia dependencies, original error: " + str(e), RuntimeWarning)


(-214.5628470363289,
 98.46305594224867,
 array([29.53076482, 33.69692161, 37.1998088 , 39.86061185, 43.9456979 ,
        46.70558317, 51.10089866]),
 102.02365200764818,
 'Thermo group additivity estimation: group(O2s-Cs(Cds-O2d)) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-CsH) + group(O2s-(Cds-O2d)H) + group(Cs-OsOsOsOs) + group(Cds-OdOsOs) + radical(OC=OOJ)')

In [13]:
resonance_radical_gav_df = pd.DataFrame()
resonance_radical_gav_df["resonance_smiles"] = hbi_df["radical_resonance_smiles"]

In [8]:
thermos = resonance_radical_gav_df["resonance_smiles"].apply(lambda smi: generate_thermo(thermo_database, smi, resonance=False))

In [21]:
temp_columns = ["H298 (kcal/mol)", "S298 (cal/mol/K)", "Cp (cal/mol/K)", "Sint298 (cal/mol/K)", "comment"]
resonance_radical_gav_df[temp_columns] = thermos.to_list()

temp_columns = []
for i, T in enumerate(Ts):
    temp_columns.append(f"Cp{T} (cal/mol/K)")
resonance_radical_gav_df[temp_columns] = resonance_radical_gav_df["Cp (cal/mol/K)"].to_list()

resonance_radical_gav_df = resonance_radical_gav_df.drop("Cp (cal/mol/K)", axis=1)

temp_columns = resonance_radical_gav_df.columns.tolist()
temp_columns.remove("comment")
temp_columns.append("comment")

resonance_radical_gav_df = resonance_radical_gav_df[temp_columns]
resonance_radical_gav_df

/home/gridsan/hwpang/mambaforge/envs/rmg_rdmc_env_20230623_v2/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,resonance_smiles,H298 (kcal/mol),S298 (cal/mol/K),Sint298 (cal/mol/K),Cp300 (cal/mol/K),Cp400 (cal/mol/K),Cp500 (cal/mol/K),Cp600 (cal/mol/K),Cp800 (cal/mol/K),Cp1000 (cal/mol/K),Cp1500 (cal/mol/K),comment
0,[O]C(=O)OC(O)(O)O,-214.562847,98.463056,102.023652,29.530765,33.696922,37.199809,39.860612,43.945698,46.705583,51.100899,Thermo group additivity estimation: group(O2s-...
1,[O]C(O)(O)OC(=O)O,-218.506442,100.780822,100.780822,28.957151,32.645296,36.363289,39.621606,44.949522,48.354723,52.797839,Thermo group additivity estimation: group(O2s-...
2,O=C(O)O[C](O)O,-182.366843,95.359992,96.737419,26.444474,30.605583,34.122333,36.909560,40.899044,43.401300,45.798700,Thermo group additivity estimation: group(O2s-...
3,CC(=O)OC(OO)C(=O)C(O)O[O],-198.122312,145.818548,145.246864,50.656616,59.598776,67.892505,73.980363,82.723365,90.133078,97.599981,Thermo group additivity estimation: group(O2s-...
4,O=[C]OC(O)(O)O,-169.366862,91.656926,93.840096,27.460975,30.914895,33.801147,36.038910,39.770268,42.499082,46.915908,Thermo group additivity estimation: group(O2s-...
...,...,...,...,...,...,...,...,...,...,...,...,...
2862,[C]1=CC=C1,156.8305,62.382873,63.7603,15.164000,18.710000,21.534000,23.741000,26.989000,29.289000,32.814000,Thermo group additivity estimation: group(Cds-...
2863,C=C=C1C=[C]C1,151.763874,52.204958,53.582385,22.076919,28.279266,33.060461,36.665145,41.822263,45.330044,50.563950,Thermo group additivity estimation: group(Cs-(...
2864,[CH2]C1=CC#CC1,160.124,71.59,71.59,22.319000,28.397000,32.609000,37.198000,43.459000,47.876000,52.869000,Thermo group additivity estimation: group(Cs-(...
2865,C=C1[CH]C#CC1,183.114,79.63,79.63,21.319000,26.687000,31.089000,35.058000,41.669000,46.126000,50.759000,Thermo group additivity estimation: group(Cs-(...


In [22]:
resonance_radical_gav_df.to_csv("../data/radical_resonance_gav.csv", index=False)